In [1]:
from accutuning_client.category import Sklearn
from accutuning_client.client import Client

In [2]:
client = Client('localhost', 8000)

In [3]:
# Experiment의 List를 불러온다.
experiments = client.experiments()
print(f'현재 Experiments는 총 {len(experiments)}개 있습니다.')
experiments

현재 Experiments는 총 2개 있습니다.


[{'id': '7',
  'name': 'Experiment-7',
  'metric': 'NEG_MEAN_SQUARED_ERROR',
  'modelsCnt': 103,
  'status': 'finished',
  'targetColumnName': 'target',
  'dataset': {'id': '13',
   'name': 'boston_ncSxlRq',
   'featureNames': ['CRIM',
    'ZN',
    'INDUS',
    'CHAS',
    'NOX',
    'RM',
    'AGE',
    'DIS',
    'RAD',
    'TAX',
    'PTRATIO',
    'B',
    'LSTAT',
    'target'],
   'processingStatus': 'FINISHED'},
  'deploymentsCnt': 0},
 {'id': '1',
  'name': 'Experiment-1',
  'metric': 'ACCURACY',
  'modelsCnt': 21,
  'status': 'finished',
  'targetColumnName': 'target',
  'dataset': {'id': '1',
   'name': 'iris',
   'featureNames': ['sepal length (cm)',
    'sepal width (cm)',
    'petal length (cm)',
    'petal width (cm)',
    'target'],
   'processingStatus': 'FINISHED'},
  'deploymentsCnt': 0}]

In [4]:
# Source List를 불러온다.
sources = client.sources()
print(f'현재 Sources는 총 {len(sources)}개 있습니다.')
sources

현재 Sources는 총 7개 있습니다.


[{'id': 38,
  'filesize': '39.2 kB',
  'filename': 'sources/boston_26MT4gw.csv',
  'fileurl': '/media/sources/boston_26MT4gw.csv',
  'datasetCount': 0,
  'createdAt': '2021-01-28T17:12:47.749677',
  'file': 'http://localhost:8000/media/sources/boston_26MT4gw.csv',
  'hidden': False,
  'samplingSize': None,
  'samplingRatio': None,
  'samplingRandomSeed': 42,
  'colNames': 'CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target',
  'colCount': 14,
  'createdBy': None},
 {'id': 37,
  'filesize': '88.8 kB',
  'filename': 'sources/diabetes2_AdcddNL.csv',
  'fileurl': '/media/sources/diabetes2_AdcddNL.csv',
  'datasetCount': 0,
  'createdAt': '2021-01-28T17:12:45.932043',
  'file': 'http://localhost:8000/media/sources/diabetes2_AdcddNL.csv',
  'hidden': False,
  'samplingSize': None,
  'samplingRatio': None,
  'samplingRandomSeed': 42,
  'colNames': 'age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target',
  'colCount': 11,
  'createdBy': None},
 {'id': 34,
  'filesize': '39.2 kB',
  'filename':

In [5]:
# Local file에서 소스를 생성한다.
client.create_source_from_file('/Users/ahaljh/Downloads/diabetes2.csv')

In [6]:
# Sklearn dataset에서 Source를 생성한다.
client.create_source_from_sklearn(Sklearn.BOSTON)

In [7]:
# Source List를 불러온다.
sources = client.sources()
print(f'현재 Sources는 총 {len(sources)}개 있습니다.')
sources

현재 Sources는 총 9개 있습니다.


[{'id': 40,
  'filesize': '39.2 kB',
  'filename': 'sources/boston_kb7s22Y.csv',
  'fileurl': '/media/sources/boston_kb7s22Y.csv',
  'datasetCount': 0,
  'createdAt': '2021-01-28T17:17:10.533092',
  'file': 'http://localhost:8000/media/sources/boston_kb7s22Y.csv',
  'hidden': False,
  'samplingSize': None,
  'samplingRatio': None,
  'samplingRandomSeed': 42,
  'colNames': 'CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target',
  'colCount': 14,
  'createdBy': None},
 {'id': 39,
  'filesize': '88.8 kB',
  'filename': 'sources/diabetes2_QUvKCb0.csv',
  'fileurl': '/media/sources/diabetes2_QUvKCb0.csv',
  'datasetCount': 0,
  'createdAt': '2021-01-28T17:17:07.328156',
  'file': 'http://localhost:8000/media/sources/diabetes2_QUvKCb0.csv',
  'hidden': False,
  'samplingSize': None,
  'samplingRatio': None,
  'samplingRandomSeed': 42,
  'colNames': 'age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target',
  'colCount': 11,
  'createdBy': None},
 {'id': 38,
  'filesize': '39.2 kB',
  'filename':

In [8]:
# Source 선택
source = sources[0]
source

{'id': 40,
 'filesize': '39.2 kB',
 'filename': 'sources/boston_kb7s22Y.csv',
 'fileurl': '/media/sources/boston_kb7s22Y.csv',
 'datasetCount': 0,
 'createdAt': '2021-01-28T17:17:10.533092',
 'file': 'http://localhost:8000/media/sources/boston_kb7s22Y.csv',
 'hidden': False,
 'samplingSize': None,
 'samplingRatio': None,
 'samplingRandomSeed': 42,
 'colNames': 'CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target',
 'colCount': 14,
 'createdBy': None}

In [9]:
# Experiment 생성 
client.create_experiment(source)

{"id":8,"status":"creating","bestScore":null,"modelsCnt":0,"deploymentsCnt":0,"doneSlot":0,"totalSlot":3600,"availableMetrics":["accuracy","balanced_accuracy","f1_macro","f1_micro","f1_weighted","neg_log_loss","precision_macro","precision_micro","precision_weighted","recall_macro","recall_micro","recall_weighted","roc_auc_ovr"],"availableEstimators":["logistic_regression","bernoulli_nb","adaboost","decision_tree","extra_trees","gradient_boosting","hist_gradient_boosting","k_nearest_neighbor","lda","liblinear_svc","libsvm_svc","nusvc","mlp","multinomial_nb","passive_aggressive","qda","random_forest","sgd","xgboost","lightgbm","catboost"],"includeEstimators":["decision_tree","gradient_boosting","libsvm_svc","lightgbm","logistic_regression","random_forest","xgboost"],"includeScalingMethods":["none","standardize","none"],"includeFeatureEngineerings":["None","SelectFwe","SelectKBest","None"],"availableScalingMethods":["standardize","minmax","normalize"],"availableFeatureEngineerings":["Sele

In [10]:
# Experiment의 List를 불러온다.
experiments = client.experiments()
print(f'현재 Experiments는 총 {len(experiments)}개 있습니다.')
experiments

현재 Experiments는 총 3개 있습니다.


[{'id': '8',
  'name': 'Experiment-8',
  'metric': 'NEG_MEAN_SQUARED_ERROR',
  'modelsCnt': 0,
  'status': 'ready',
  'targetColumnName': 'target',
  'dataset': {'id': '15',
   'name': 'boston_kb7s22Y',
   'featureNames': ['CRIM',
    'ZN',
    'INDUS',
    'CHAS',
    'NOX',
    'RM',
    'AGE',
    'DIS',
    'RAD',
    'TAX',
    'PTRATIO',
    'B',
    'LSTAT',
    'target'],
   'processingStatus': 'READY'},
  'deploymentsCnt': 0},
 {'id': '7',
  'name': 'Experiment-7',
  'metric': 'NEG_MEAN_SQUARED_ERROR',
  'modelsCnt': 103,
  'status': 'finished',
  'targetColumnName': 'target',
  'dataset': {'id': '13',
   'name': 'boston_ncSxlRq',
   'featureNames': ['CRIM',
    'ZN',
    'INDUS',
    'CHAS',
    'NOX',
    'RM',
    'AGE',
    'DIS',
    'RAD',
    'TAX',
    'PTRATIO',
    'B',
    'LSTAT',
    'target'],
   'processingStatus': 'FINISHED'},
  'deploymentsCnt': 0},
 {'id': '1',
  'name': 'Experiment-1',
  'metric': 'ACCURACY',
  'modelsCnt': 21,
  'status': 'finished',
  'tar

In [11]:
# Experiment 선택
experiment = experiments[0]
experiment

{'id': '8',
 'name': 'Experiment-8',
 'metric': 'NEG_MEAN_SQUARED_ERROR',
 'modelsCnt': 0,
 'status': 'ready',
 'targetColumnName': 'target',
 'dataset': {'id': '15',
  'name': 'boston_kb7s22Y',
  'featureNames': ['CRIM',
   'ZN',
   'INDUS',
   'CHAS',
   'NOX',
   'RM',
   'AGE',
   'DIS',
   'RAD',
   'TAX',
   'PTRATIO',
   'B',
   'LSTAT',
   'target'],
  'processingStatus': 'READY'},
 'deploymentsCnt': 0}

In [12]:
# Run Automl
if experiment.get('status') == 'ready':
    # Preprocess before a Run
    client.preprocessor_config_recommend(experiment)
    client.preprocess(experiment)
    print('Run AutoML')
    client.run(experiment)
else:
    print('실행 가능한 상태가 아님')

Run AutoML


In [13]:
# 시간이 오래 걸리니까 완료된 experiment를 다시 선택
experiment = experiments[1]
experiment

{'id': '7',
 'name': 'Experiment-7',
 'metric': 'NEG_MEAN_SQUARED_ERROR',
 'modelsCnt': 103,
 'status': 'finished',
 'targetColumnName': 'target',
 'dataset': {'id': '13',
  'name': 'boston_ncSxlRq',
  'featureNames': ['CRIM',
   'ZN',
   'INDUS',
   'CHAS',
   'NOX',
   'RM',
   'AGE',
   'DIS',
   'RAD',
   'TAX',
   'PTRATIO',
   'B',
   'LSTAT',
   'target'],
  'processingStatus': 'FINISHED'},
 'deploymentsCnt': 0}

In [14]:
# Leaderboard 정보 구해오기 
leaderboard = client.leaderboard(experiment)
print(f'leaderboad의 model 갯수는 {len(leaderboard)}')
leaderboard

leaderboad의 model 갯수는 103


[{'id': '551',
  'score': -7.232996463775635,
  'trainScore': -0.19474734365940094,
  'validScore': -7.232996463775635,
  'testScore': 0.0,
  'estimatorName': 'xgboost',
  'generator': 'optuna-pycaret',
  'file': {'size': '304137', 'sizeHumanized': '304.1 kB'},
  'deployedStatus': None},
 {'id': '561',
  'score': -7.340738773345947,
  'trainScore': -0.439432829618454,
  'validScore': -7.340738773345947,
  'testScore': 0.0,
  'estimatorName': 'xgboost',
  'generator': 'optuna-pycaret',
  'file': {'size': '236764', 'sizeHumanized': '236.8 kB'},
  'deployedStatus': None},
 {'id': '562',
  'score': -7.340738773345947,
  'trainScore': -0.439432829618454,
  'validScore': -7.340738773345947,
  'testScore': 0.0,
  'estimatorName': 'xgboost',
  'generator': 'optuna-pycaret',
  'file': {'size': '236764', 'sizeHumanized': '236.8 kB'},
  'deployedStatus': None},
 {'id': '563',
  'score': -7.340738773345947,
  'trainScore': -0.439432829618454,
  'validScore': -7.340738773345947,
  'testScore': 0.0,

In [15]:
# 가장 순위 높은 모델 선택
model = leaderboard[0]
model

{'id': '551',
 'score': -7.232996463775635,
 'trainScore': -0.19474734365940094,
 'validScore': -7.232996463775635,
 'testScore': 0.0,
 'estimatorName': 'xgboost',
 'generator': 'optuna-pycaret',
 'file': {'size': '304137', 'sizeHumanized': '304.1 kB'},
 'deployedStatus': None}

In [16]:
# 모델 배포
if model.get('deployedStatus') is None:
    print('deploy model')
    # 모델 배포
    client.deploy(model)
else:
    print('모델 배포가 가능한 상태가 아님')

deploy model


In [17]:
# Deployment 정보 구해오기 
deployments = client.deployments(experiment)
print(f'배포된 모델은 {len(deployments)}개 입니다.')
deployed_model = deployments[0]
deployed_model

{'deployments': [{'id': '5', 'name': 'xgboost', 'description': None, 'status': 'DONE', 'modelType': 'model', 'modelPk': 551, 'allMetricsJson': None, 'createdAt': '2021-01-28T17:18:03.756555', 'testScore': None, 'model': {'id': '551', 'trainScore': -0.19474734365940094, 'validScore': -7.232996463775635}, 'file': {'url': 'http://localhost:8000/media/runtime_0007/runtimeprocess_0049/output/pipeline.pkl', 'size': '304117', 'sizeHumanized': '304.1 kB', 'name': 'runtime_0007/runtimeprocess_0049/output/pipeline.pkl'}}]}
배포된 모델은 1개 입니다.


{'id': '5',
 'name': 'xgboost',
 'description': None,
 'status': 'DONE',
 'modelType': 'model',
 'modelPk': 551,
 'allMetricsJson': None,
 'createdAt': '2021-01-28T17:18:03.756555',
 'testScore': None,
 'model': {'id': '551',
  'trainScore': -0.19474734365940094,
  'validScore': -7.232996463775635},
 'file': {'url': 'http://localhost:8000/media/runtime_0007/runtimeprocess_0049/output/pipeline.pkl',
  'size': '304117',
  'sizeHumanized': '304.1 kB',
  'name': 'runtime_0007/runtimeprocess_0049/output/pipeline.pkl'}}

In [18]:
# 모델 예측을 위해 Default값인 최빈값을 구함
most_frequent = client.mostfrequent(experiment)
most_frequent

[{'id': '151',
  'name': 'CRIM',
  'datatype': 'FLOAT64',
  'mostFrequent': '14.3337',
  'min': 0.01,
  'max': 88.98},
 {'id': '152',
  'name': 'ZN',
  'datatype': 'FLOAT64',
  'mostFrequent': '0.0',
  'min': 0.0,
  'max': 100.0},
 {'id': '153',
  'name': 'INDUS',
  'datatype': 'FLOAT64',
  'mostFrequent': '18.1',
  'min': 0.46,
  'max': 27.74},
 {'id': '154',
  'name': 'CHAS',
  'datatype': 'FLOAT64',
  'mostFrequent': '0.0',
  'min': 0.0,
  'max': 1.0},
 {'id': '155',
  'name': 'NOX',
  'datatype': 'FLOAT64',
  'mostFrequent': '0.5379999999999999',
  'min': 0.38,
  'max': 0.87},
 {'id': '156',
  'name': 'RM',
  'datatype': 'FLOAT64',
  'mostFrequent': '6.405',
  'min': 3.56,
  'max': 8.78},
 {'id': '157',
  'name': 'AGE',
  'datatype': 'FLOAT64',
  'mostFrequent': '100.0',
  'min': 2.9,
  'max': 100.0},
 {'id': '158',
  'name': 'DIS',
  'datatype': 'FLOAT64',
  'mostFrequent': '3.4952',
  'min': 1.13,
  'max': 12.13},
 {'id': '159',
  'name': 'RAD',
  'datatype': 'FLOAT64',
  'mostFr

In [19]:
# 예측할 input값을 만듬(최빈값 사용) 
input_val = {col.get('name'): col.get('mostFrequent') for col in most_frequent}
input_val

{'CRIM': '14.3337',
 'ZN': '0.0',
 'INDUS': '18.1',
 'CHAS': '0.0',
 'NOX': '0.5379999999999999',
 'RM': '6.405',
 'AGE': '100.0',
 'DIS': '3.4952',
 'RAD': '24.0',
 'TAX': '666.0',
 'PTRATIO': '20.2',
 'B': '396.9',
 'LSTAT': '14.1'}

In [20]:
# 예측
predict_val = client.predict(deployed_model, input_val, experiment.get('id'))
print(f'예측값은 {predict_val} 입니다.')

예측값은 19.94942283630371 입니다.
